In [26]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [102]:
import numpy as np

def abc_algorithm(num_employed, num_onlooker, num_scout, iterations, dimension,
                  lower_bound, upper_bound, f, task='min'):
    
    def safe_eval(x):
        """Ensures the output of f(x) is a scalar float."""
        result = f(x)
        if isinstance(result, (tuple, list, np.ndarray)):
            return float(result[0])
        return float(result)

    def calculate_fitness(values):
        """Convert raw objective values into fitness values."""
        values = np.array(values)
        if task == 'max':
            return 1 / (1 + np.max(values) - values)
        else:
            return 1 / (1 + np.clip(values, 0, None))  # prevent negative/zero division

    # Initialization
    total_bees = num_employed + num_onlooker + num_scout
    population = np.random.uniform(lower_bound, upper_bound, size=(total_bees, dimension))
    fitness_values = np.array([safe_eval(ind) for ind in population])
    fitness = calculate_fitness(fitness_values)
    trials = np.zeros(total_bees)

    # Best solution so far
    best_idx = np.argmax(fitness) if task == 'max' else np.argmin(fitness_values)
    best_solution = population[best_idx].copy()
    best_value = fitness_values[best_idx]

    for _ in range(iterations):
        # === Employed Bee Phase ===
        for i in range(num_employed):
            k = np.random.randint(total_bees)
            while k == i:
                k = np.random.randint(total_bees)
            phi = np.random.uniform(-1, 1, size=dimension)
            candidate = np.clip(population[i] + phi * (population[i] - population[k]), lower_bound, upper_bound)
            candidate_value = safe_eval(candidate)

            if (task == 'max' and candidate_value > fitness_values[i]) or (task == 'min' and candidate_value < fitness_values[i]):
                population[i] = candidate
                fitness_values[i] = candidate_value
                fitness[i] = calculate_fitness([candidate_value])[0]
                trials[i] = 0
            else:
                trials[i] += 1

        # === Onlooker Bee Phase ===
        prob = fitness[:num_employed]
        prob_sum = np.sum(prob)
        prob = prob / prob_sum if prob_sum > 0 else np.ones(num_employed) / num_employed

        for _ in range(num_onlooker):
            i = np.random.choice(num_employed, p=prob)
            k = np.random.randint(total_bees)
            while k == i:
                k = np.random.randint(total_bees)
            phi = np.random.uniform(-1, 1, size=dimension)
            candidate = np.clip(population[i] + phi * (population[i] - population[k]), lower_bound, upper_bound)
            candidate_value = safe_eval(candidate)

            if (task == 'max' and candidate_value > fitness_values[i]) or (task == 'min' and candidate_value < fitness_values[i]):
                population[i] = candidate
                fitness_values[i] = candidate_value
                fitness[i] = calculate_fitness([candidate_value])[0]
                trials[i] = 0
            else:
                trials[i] += 1

        # === Scout Bee Phase ===
        limit = iterations / 2
        for i in range(total_bees):
            if trials[i] > limit:
                population[i] = np.random.uniform(lower_bound, upper_bound, size=dimension)
                fitness_values[i] = safe_eval(population[i])
                fitness[i] = calculate_fitness([fitness_values[i]])[0]
                trials[i] = 0

        # === Update Best ===
        best_idx = np.argmax(fitness) if task == 'max' else np.argmin(fitness_values)
        if (task == 'max' and fitness_values[best_idx] > best_value) or (task == 'min' and fitness_values[best_idx] < best_value):
            best_solution = population[best_idx].copy()
            best_value = fitness_values[best_idx]

    return best_solution, best_value


In [103]:
def beale(X):
    return ((1.5 - X[0] + (X[0]*X[1]))**2)  + ((2.25 - X[0]+(X[0]*(X[1]**2)))**2)+ ((2.625-X[0]+(X[0]*(X[1]**3)))**2)

## valor optimo 3,.5, f = 0
beale([3, 0.5])

0.0

In [104]:
from timeit import default_timer as timer

In [105]:
start = timer()
# num_employed, num_onlooker, num_scout, iterations, dimension, lower_bound, upper_bound, f, task='min'
print(abc_algorithm(150, 150, 150, 150,2, -100, 100, beale, task='min'))
stop = timer()
execution_time = stop - start
print(execution_time)

(array([3.00906837, 0.50208035]), np.float64(1.362779970686686e-05))
3.7928364999970654


In [106]:
def McCormick(X):
    return np.sin(X[0]+X[1]) + (X[0]-X[1])**2 - 1.5 * X[0] + 2.5 * X[1] + 1

In [107]:
McCormick([-0.54719,-1.54719])

np.float64(-1.913222954882274)

In [108]:
start = timer()
print(abc_algorithm(100, 150, 150, 150,2, -3, 4, McCormick, task='min'))
stop = timer()
execution_time = stop - start
print(execution_time)

(array([-0.54719622, -1.54719722]), np.float64(-1.9132229549788335))
3.470028199997614


In [109]:
def himm(X):
    return ((X[0]**2+X[1]-11)**2) + ((X[0]+(X[1]**2)-7)**2 )
himm([-0.270845,-0.923039])

181.61652152257642

In [111]:
start = timer()
# num_employed, num_onlooker, num_scout, iterations, dimension, lower_bound, upper_bound, f, task='min'
print(abc_algorithm(150, 150, 150, 150,2, -1, 1, himm, task='max'))
stop = timer()
execution_time = stop - start
print(execution_time)

(array([-0.27084462, -0.9230386 ]), np.float64(181.61652152258267))
3.5402315000028466
